In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!git clone https://github.com/Florisbruinsma/GoGame

Cloning into 'GoGame'...
remote: Enumerating objects: 156, done.
remote: Counting objects: 100% (156/156), done.
remote: Compressing objects: 100% (144/144), done.
remote: Total 156 (delta 82), reused 75 (delta 6), pack-reused 0
Receiving objects: 100% (156/156), 86.28 KiB | 9.59 MiB/s, done.
Resolving deltas: 100% (82/82), done.


In [4]:
%tensorflow_version 2.x
import tensorflow as tf
print(tf.__version__)
print(tf.test.is_gpu_available())

from GoGame.GoGame import GoGame
from GoGame.Model import Model
from GoGame.Agent import A2CAgent

# import gym
%load_ext tensorboard
from tensorflow.keras import backend
from tensorflow.keras import callbacks
import os
import time
import numpy as np
# %tensorboard — logdir logs

TensorFlow 2.x selected.
2.0.0
True


In [5]:
EPOCHS = 20
EPISODES = 200
BOARDSIZE = 5
MAXTURNS = (BOARDSIZE*BOARDSIZE)-BOARDSIZE
model_name = 'goGameV1Size'+str(BOARDSIZE)
path = "/content/drive/My Drive/colab/saved_model_data/"
save_path = path + "{}.h5".format(model_name)
log_dir = path + "/logs"
tensorboard_callback = callbacks.TensorBoard(log_dir, histogram_freq=1,write_images=True,write_grads=True,write_graph=True,update_freq='batch')
callbacks = [tensorboard_callback]

In [0]:
%tensorboard --logdir "/content/drive/My Drive/colab/saved_model_data/logs/train"

In [8]:
env = GoGame(BOARDSIZE, maxTurns=MAXTURNS)

model = Model(num_actions=env.action_space.n, callbacks=callbacks)
agent = A2CAgent(model)
agent.test(env,render=False)

if(os.path.isfile(save_path)):
    model.load_weights(save_path)
    print("model loaded")
else:
    print("new model")


for epoch in range(EPOCHS):
    time_start = time.time()
    backend.clear_session()
    rewards_history, episode_wins = agent.train(env,max_steps=MAXTURNS,episodes=EPISODES, info=False, info_step=50)
    print("epoch = {:2} | won {:3}/{:3} matches | mean rewards = {:7.2f} | epoch time = {:6.2f} sec".format(epoch, episode_wins.count(1), len(episode_wins), np.mean(rewards_history), (time.time()-time_start)))
    model.save_weights(save_path)
    print("model saved")

# print("Finished training, testing...")
# for _ in range(5):
#     print("total score = %d " % agent.test(env,render=True))

TypeError: ignored